# Descrizione notebook
In questo notebook si fa utilizzo dell'Item-Based Collaborative Filtering per raccomandare una barzelletta ad un utente.

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd
from itertools import combinations
import math

In [2]:
# lettura dei rating normalizzati dal relativo CSV
normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')

# lettura dei rating non normalizzati dal relativo CSV
ratings_df = pd.read_csv('./data/jester_jokes_ratings.csv')

# Similarità tra item: Adjusted Coseno Similarità

In [3]:
def cosin_similarity(x, y): 
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        t1+=i*j # numeratore
        t2+=i*i # denominatore_1
        t3+=j*j # denominatore_2
    return t1 / (np.sqrt(t2) * np.sqrt(t3))

# Predizione

In [4]:
def score_user_item(item_target, items_rated, active_user_rating, df):
    t1, t2 = 0, 0
    for i in items_rated:
        sim = float(df.loc[((df['elem_1'] == item_target) & (df['elem_2'] == i)) | ((df['elem_1'] == i) & (df['elem_2'] == item_target))]["similarity"])
        t1+= sim * float(active_user_rating[i]) # numeratore
        t2+= sim # denominatore
    score = t1 / t2
    return score

# Raccomandazione

In [5]:
# si costruisce la matrice di similarità, ovvero una lista di coppie di item, per 
# ogni coppia si specifica la similarità (ad -1 a 1)
all_jokes = (list(normalized_ratings_df.columns))[2:]
jokes_pairs = list(combinations(all_jokes, 2))

similarity_matrix = []

for i_1, i_2 in jokes_pairs:
    app = normalized_ratings_df[[i_1, i_2]].dropna()
    similarity_matrix.append([i_1, i_2, cosin_similarity(app[i_1], app[i_2])])
    
df = pd.DataFrame(similarity_matrix, columns = ['elem_1', 'elem_2', 'similarity'])
df.head()

,elem_1,elem_2,similarity
0,joke_1,joke_2,0.202696
1,joke_1,joke_3,0.173774
2,joke_1,joke_4,0.018405
3,joke_1,joke_5,0.006122
4,joke_1,joke_6,-0.086289


In [6]:
def recommendation_2(active_user_id, ratings_df, df):
    # si prende la riga relativa all'utente con l'id specificato
    active_user = ratings_df[ratings_df['user_id'] == active_user_id]
    # si considerano solo i rating espressi dall'utente target
    active_user_rating = active_user.iloc[:, 2:]
    # si prendono le barzellette raccomandabili all'utente
    recommendation_columns = []
    for column in active_user_rating.columns:
        if math.isnan(active_user_rating[column].values[0]):
            recommendation_columns.append(column)

    # si prendono le barzellette già valutate dall'utente
    rated_items = list(set(list(active_user_rating.columns)) - set(recommendation_columns))
    # si fa la raccomandazione
    top_score = -np.inf
    joke_to_suggest = ''
    final = []
    for joke in recommendation_columns:
        score = score_user_item(joke, rated_items, active_user_rating, df)
        print("per la barzelletta " + joke + " si è predetto un punteggio di: " + str(score))
        if score > top_score:
            top_score = score
            joke_to_suggest = joke
        app_2 = {}
        app_2["user_id"] = active_user_id
        app_2[joke] = score
        final.append(app_2)
    print('\nil valore più alto predetto è ' + str(top_score) + ' su 10')
    print('quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è: ', joke_to_suggest)
    return final
    

In [8]:
recommendation_2(1001, ratings_df)

per la barzelletta joke_1 si è predetto un punteggio di: 5.48918911153894
per la barzelletta joke_2 si è predetto un punteggio di: 5.874896885310737
per la barzelletta joke_3 si è predetto un punteggio di: 5.530364213231964
per la barzelletta joke_4 si è predetto un punteggio di: 5.6830206651215
per la barzelletta joke_9 si è predetto un punteggio di: 5.629541237464792
per la barzelletta joke_23 si è predetto un punteggio di: 5.580920494036714
per la barzelletta joke_24 si è predetto un punteggio di: 5.783086061131673
per la barzelletta joke_33 si è predetto un punteggio di: 5.681675853663834
per la barzelletta joke_37 si è predetto un punteggio di: 5.920645912010234
per la barzelletta joke_43 si è predetto un punteggio di: 5.719735411098292
per la barzelletta joke_47 si è predetto un punteggio di: 8.07166285418944
per la barzelletta joke_55 si è predetto un punteggio di: 5.573499141349483
per la barzelletta joke_57 si è predetto un punteggio di: 5.92393050044584
per la barzelletta jok

[{'user_id': 1001, 'joke_1': 5.48918911153894},
 {'user_id': 1001, 'joke_2': 5.874896885310737},
 {'user_id': 1001, 'joke_3': 5.530364213231964},
 {'user_id': 1001, 'joke_4': 5.6830206651215},
 {'user_id': 1001, 'joke_9': 5.629541237464792},
 {'user_id': 1001, 'joke_23': 5.580920494036714},
 {'user_id': 1001, 'joke_24': 5.783086061131673},
 {'user_id': 1001, 'joke_33': 5.681675853663834},
 {'user_id': 1001, 'joke_37': 5.920645912010234},
 {'user_id': 1001, 'joke_43': 5.719735411098292},
 {'user_id': 1001, 'joke_47': 8.07166285418944},
 {'user_id': 1001, 'joke_55': 5.573499141349483},
 {'user_id': 1001, 'joke_57': 5.92393050044584},
 {'user_id': 1001, 'joke_58': 5.979317271578034},
 {'user_id': 1001, 'joke_60': 5.916319952555862},
 {'user_id': 1001, 'joke_64': 5.9759776161004226},
 {'user_id': 1001, 'joke_67': 5.726215401671431},
 {'user_id': 1001, 'joke_71': 5.4738985269520235},
 {'user_id': 1001, 'joke_72': 9.827958290733303},
 {'user_id': 1001, 'joke_73': 4.587967997499567},
 {'user_